In [1]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import pandas as pd
import requests
from io import StringIO
from tqdm import tqdm
from modlamp.descriptors import GlobalDescriptor


In [2]:
def fetch_uniprot_batches(query="reviewed:true", batch_size=500, total=10000):
    base_url = "https://rest.uniprot.org/uniprotkb/search"
    all_dataframes = []
    fetched = 0
    params = {
        "query": query,
        "format": "tsv",
        "fields": "accession,id,protein_name,organism_name,length,sequence,mass,cc_subcellular_location,go_p,go_f,ec,cc_disruption_phenotype,cc_catalytic_activity,cc_pathway,cc_subcellular_location,cc_function,cc_domain,cc_induction,cc_pharmaceutical,cc_disruption_phenotype",
        "size": batch_size,
    }

    response = requests.get(base_url, params=params)
    if not response.ok:
        raise Exception(f"Initial fetch failed: {response.status_code} {response.text}")

    with tqdm(total=total, desc="Fetching UniProt entries") as pbar:
        while True:
            df_batch = pd.read_csv(StringIO(response.text), sep="\t")
            if df_batch.empty:
                break

            all_dataframes.append(df_batch)
            batch_size = len(df_batch)
            fetched += batch_size
            pbar.update(batch_size)

            if fetched >= total:
                break

          # Get full next URL from response.links
            next_url = response.links.get("next", {}).get("url", None)
            if not next_url:
                break

            response = requests.get(next_url)
            if not response.ok:
                raise Exception(f"Next fetch failed: {response.status_code} {response.text}")

    return pd.concat(all_dataframes, ignore_index=True)



In [3]:
def compute_biopython_features(seq: str, entry: str) -> dict:
    analysis = ProteinAnalysis(seq)

    try:
        aa_counts = analysis.count_amino_acids()
        aa_percents = analysis.get_amino_acids_percent()
        sec_frac = analysis.secondary_structure_fraction()
    except Exception as e:
        print(f"Failed on sequence: {seq} - {e}")
        return None

    try:
        features = {
            "Entry": entry,
            "Sequence": seq,
            "length": len(seq),
            "mol_weight": analysis.molecular_weight(),
            "iso_point": analysis.isoelectric_point(),
            "aromaticity": analysis.aromaticity(),
            "instability_index": analysis.instability_index(),
            "gravy": analysis.gravy(),
            "helix_frac": sec_frac[0],
            "turn_frac": sec_frac[1],
            "sheet_frac": sec_frac[2],
        }
    except Exception as e:
        print(f"Failed to compute features for sequence: {seq} - {e}")
        return None

    # Add amino acid counts and percents
    for aa in aa_counts:
        features[f"count_{aa}"] = aa_counts[aa]
        features[f"percent_{aa}"] = round(aa_percents[aa],3)

    
    # --- modlamp GlobalDescriptor ---
    try:
        desc = GlobalDescriptor([seq])
        desc.calculate_all()
        modlamp_feats = desc.descriptor[0]
        modlamp_names = [
            "charge_pH7", "boman_index", "aliphatic_index", "hydrophobic_moment"
        ]

        for name, value in zip(modlamp_names, modlamp_feats):
            orig_name = name.split("_", 1)[-1]
            features[name] = value

    except Exception as e:
        print(f"modlamp failed on sequence: {seq} - {e}")
        return None

    return features


In [4]:
df = fetch_uniprot_batches(total=10000)
df_filtered = df[df['Length'] <= 1024]


Fetching UniProt entries: 100%|██████████| 10000/10000 [00:47<00:00, 211.30it/s]


In [5]:
sequences = df["Sequence"].tolist()

feature_dicts = []
for seq in tqdm(sequences):
    entry = df.loc[df["Sequence"] == seq, "Entry"].values[0]
    #print(entry)
    features = compute_biopython_features(seq, entry)
    if features is not None:
        feature_dicts.append(features)

  0%|          | 0/10000 [00:00<?, ?it/s]/home/ubuntu/.local/lib/python3.10/site-packages/Bio/SeqUtils/ProtParam.py:106: BiopythonDeprecationWarning: The get_amino_acids_percent method has been deprecated and will likely be removed from Biopython in the near future. Please use the amino_acids_percent attribute instead.
  warnings.warn(
 11%|█         | 1085/10000 [00:01<00:11, 776.35it/s]

Failed to compute features for sequence: MGLPQPGLWLKRLWVLLEVAVHVVVGKVLLILFPDRVKRNILAMGEKTGMTRNPHFSHDNWIPTFFSTQYFWFVLKVRWQRLEDTTELGGLAPNCPVVRLSGQRCNIWEFMQGNRPLVLNFGSCTUPSFMFKFDQFKRLIEDFSSIADFLVIYIEEAHASDGWAFKNNMDIRNHQNLQDRLQAAHLLLARSPQCPVVVDTMQNQSSQLYAALPERLYIIQEGRILYKGKSGPWNYNPEEVRAVLEKLHS - 'U'
Failed to compute features for sequence: MWRSLGLALALCLLPSGGTESQDQSSLCKQPPAWSIRDQDPMLNSNGSVTVVALLQASUYLCILQASKLEDLRVKLKKEGYSNISYIVVNHQGISSRLKYTHLKNKVSEHIPVYQQEENQTDVWTLLNGSKDDFLIYDRCGRLVYHLGLPFSFLTFPYVEEAIKIAYCEKKCGNCSLTTLKDEDFCKRVSLATVDKTVETPSPHYHHEHHHNHGHQHLGSSELSENQQPGAPNAPTHPAPPGLHHHHKHKGQHRQGHPENRDMPASEDLQDLQKKLCRKRCINQLLCKLPTDSELAPRSUCCHCRHLIFEKTGSAITUQCKENLPSLCSUQGLRAEENITESCQURLPPAAUQISQQLIPTEASASURUKNQAKKUEUPSN - 'U'


 25%|██▌       | 2502/10000 [00:03<00:09, 751.28it/s]

Failed to compute features for sequence: MAGYEYVSPEQLAGFDKYKYSAVDTNPLSLYVMHPFWNTIVKVFPTWLAPNLITFSGFLLVVFNFLLMAYFDPDFYASAPGHKHVPDWVWIVVGILNFVAYTLDGVDGKQARRTNSSTPLGELFDHGLDSWSCVYFVVTVYSIFGRGSTGVSVFVLYLLLWVVLFSFILSHWEKYNTGILFLPWGYDISQVTISFVYIVTAVVGVEAWYEPFLFNFLYRDLFTAMIIGCALCVTLPMSLLNFFRSYKNNTLKLNSVYEAMVPLFSPCLLFILSTAWILWSPSDILELHPRVFYFMVGTAFANSTCQLIVCQMSSTRCPTLNWLLVPLFLVVLVVNLGVASYVESILLYTLTTAFTLAHIHYGVRVVKQLSSHFQIYPFSLRKPNSDULGMEEKNIGL - 'U'


 27%|██▋       | 2659/10000 [00:03<00:09, 757.89it/s]

...

 73%|███████▎  | 7306/10000 [00:10<00:03, 728.41it/s]

Failed to compute features for sequence: MGCAEGKAVAAAAPTELQTKGKNGDGRRRSAKDHHPGKTLPENPAGFTSTATADSRALLQAYIDGHSVVIFSRSTCTRCTEVKKLFKSLCVPYFVLELDQTEDGRALEGTLSELAAETDLPVVFVKQRKIGGHGPTLKAYQEGRLQKLLKMNGPEDLPKSYDYDLIIIGGGSGGLAAAKEAAQYGKKVMVLDFVTPTPLGTRWGLGGTCVNVGCIPKKLMHQAALLGQALQDSRNYGWKVEETVKHDWDRMIEAVQNHIGSLNWGYRVALREKKVVYENAYGQFIGPHRIKATNNKGKEKIYSAERFLIATGERPRYLGIPGDKEYCISSDDLFSLPYCPGKTLVVGASYVALECAGFLAGIGLDVTVMVRSILLRGFDQDMANKIGEHMEEHGIKFIRQFVPIKVEQIEAGTPGRLRVVAQSTNSEEIIEGEYNTVMLAIGRDACTRKIGLETVGVKINEKTGKIPVTDEEQTNVPYIYAIGDILEDKVELTPVAIQAGRLLAQRLYAGSTVKCDYENVPTTVFTPLEYGACGLSEEKAVEKFGEENIEVYHSYFWPLEWTIPSRDNNKCYAKIICNTKDNERVVGFHVLGPNAGEVTQGFAAALKCGLTKKQLDSTIGIHPVCAEVFTTLSVTKRSGASILQAGCUG - 'U'


 83%|████████▎ | 8276/10000 [00:11<00:02, 756.77it/s]

Failed to compute features for sequence: MERQEESLSARPALETEGLRFLHTTVGSLLATYGWYIVFSCILLYVVFQKLSARLRALRQRQLDRAAAAVEPDVVVKRQEALAAARLKMQEELNAQVEKHKEKLKQLEEEKRRQKIEMWDSMQEGKSYKGNAKKPQEEDSPGPSTSSVLKRKSDRKPLRGGGYNPLSGEGGGACSWRPGRRGPSSGGUG - 'U'
Failed to compute features for sequence: MAVYRAALGASLAAARLLPLGRCSPSPAPRSTLSGAAMEPAPRWLAGLRFDNRALRALPVEAPPPGPEGAPSAPRPVPGACFTRVQPTPLRQPRLVALSEPALALLGLGAPPAREAEAEAALFFSGNALLPGAEPAAHCYCGHQFGQFAGQLGDGAAMYLGEVCTATGERWELQLKGAGPTPFSRQADGRKVLRSSIREFLCSEAMFHLGVPTTRAGACVTSESTVVRDVFYDGNPKYEQCTVVLRVASTFIRFGSFEIFKSADEHTGRAGPSVGRNDIRVQLLDYVISSFYPEIQAAHASDSVQRNAAFFREVTRRTARMVAEWQCVGFCHGVLNTDNMSILGLTIDYGPFGFLDRYDPDHVCNASDNTGRYAYSKQPEVCRWNLRKLAEALQPELPLELGEAILAEEFDAEFQRHYLQKMRRKLGLVQVELEEDGALVSKLLETMHLTGADFTNTFYLLSSFPVELESPGLAEFLARLMEQCASLEELRLAFRPQMDPRQLSMMLMLAQSNPQLFALMGTRAGIARELERVEQQSRLEQLSAAELQSRNQGHWADWLQAYRARLDKDLEGAGDAAAWQAEHVRVMHANNPKYVLRNYIAQNAIEAAERGDFSEVRRVLKLLETPYHCEAGAATDAEATEADGADGRQRSYSSKPPLWAAELCVTUSS - 'U'


100%|██████████| 10000/10000 [00:13<00:00, 730.58it/s]


In [6]:
master_df = pd.DataFrame(feature_dicts)
master_master_df = master_df.merge(df, on="Sequence", how="left")
master_master_df = master_master_df.rename(columns={"Entry_x": "Entry"})


In [8]:
#save to csv
master_master_df.to_csv("data_collector.csv", index=False)
